<a href="https://colab.research.google.com/github/baeksumin/k-project/blob/main/data_preprocessing/optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GDC(Golden Dead Cross)

In [1]:
!pip install backtrader

     |████████████████████████████████| 410 kB 38.7 MB/s 


In [2]:
pip install finance-datareader

In [3]:
pip install backtesting

     |████████████████████████████████| 174 kB 28.6 MB/s 
  Created wheel for backtesting: filename=Backtesting-0.3.2-py3-none-any.whl size=173671 sha256=eb4faa547104e00d377d8cfad3d38a929e8136cd0c1f5244a16efb762db440fe
  Stored in directory: /root/.cache/pip/wheels/22/0e/30/e458272563bdbea1905391b5bb5702e2001a769544d9cb2df3
Successfully built backtesting


In [4]:
pip install deap

     |████████████████████████████████| 160 kB 32.6 MB/s 


In [30]:
#talib 설치
!wget -q http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz 2>&1 > /dev/null
!tar xvzf ta-lib-0.4.0-src.tar.gz 2>&1 > /dev/null

import os

os.chdir('ta-lib')  

!./configure --prefix=/usr 2>&1 > /dev/null
!make 2>&1 > /dev/null
!make install 2>&1 > /dev/null

os.chdir('../')

!pip install TA-Lib 2>&1 > /dev/null

./configure: line 4354: /usr/bin/file: No such file or directory
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
ar: `u' modifier ignored since `D' is the default (see `U')
gen_code.c: In function ‘printFuncHeaderDoc’:
gen_code.c:3456:4: warning: format not a string literal and no format arguments [-Wformat-security]
    fprintf( out, prefix );
    ^~~~~~~


In [31]:
import re
import json
import talib
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, date, timedelta
from google.colab import drive
import backtrader as bt
from IPython.display import display, Image
import backtrader.feeds as btfeeds
from backtesting import Strategy
from backtesting import Backtest
from deap import base, creator, tools, algorithms
import random
import seaborn as sns
from tqdm import trange
import time

In [6]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'


In [7]:
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/[Shared]K-Project/data/'

Mounted at /content/drive


In [8]:
## Load pickle - fullcode 목록
with open(data_path + 'full_code.pkl',"rb") as f: 
    fullcode = pickle.load(f)
    
# 상장폐지 상태 제외시키기
fullcode_filter = fullcode[fullcode['status'] != '상장폐지']

# fullcode열을 list로 변경
fullcode_list = list(fullcode_filter['full_code'])
print(fullcode_list[:10])
print(len(fullcode_list))

['KR7060310000', 'KR7095570008', 'KR7006840003', 'KR7054620000', 'KR7265520007', 'KR7211270004', 'KR7027410000', 'KR7282330000', 'KR7138930003', 'KR7001460005']
2603


In [9]:
with open(data_path + 'initial_data1.json',"r") as json_file:
    json_data = json.load(json_file) 

In [35]:
# 시가총액 상위 종목
mktcap_code = ['KR7005930003', 'KR7000660001', 'KR7035420009']#, 'KR7207940008', 'KR7005931001', 'KR7035720002', 'KR7051910008', 'KR7006400006', 'KR7005380001', 'KR7000270009']
mktcap_code

['KR7005930003', 'KR7000660001', 'KR7035420009']

In [12]:
for i in mktcap_code:  
  
  # backtest 데이터 가공

  data = pd.read_json(json_data[i], orient ='index') 
  df = data.transpose()

  #시간순 재정렬
  df = df.sort_values(by=['TRD_DD'])
  df.reset_index(drop=True,inplace=True)
  df['TRD_DD'] = pd.to_datetime(df['TRD_DD']) #datetime변환

  #인풋 데이터 모양 맞춰주기(backtest에 들어갈 데이터 모양)
  df_bt = df[['TRD_DD','TDD_OPNPRC','TDD_HGPRC','TDD_LWPRC','TDD_CLSPRC', 'ACC_TRDVOL']]
  df_bt['TRD_DD'] = pd.to_datetime(df_bt['TRD_DD'])
  df_bt.rename(columns={'TRD_DD':'Date', 'TDD_OPNPRC':'Open', 'TDD_HGPRC':'High','TDD_LWPRC':'Low','TDD_CLSPRC':'Close', 'ACC_TRDVOL':'Volume'}, inplace=True)
  df_bt.set_index('Date',drop=True,inplace=True)

  #데이터프레임 콤마(,) 제거 그리고 타입 소수로 변환
  df_bt['Open'] = df_bt['Open'].str.replace(',','').astype('float')
  df_bt['High'] = df_bt['High'].str.replace(',','').astype('float')
  df_bt['Low'] = df_bt['Low'].str.replace(',','').astype('float')
  df_bt['Close'] = df_bt['Close'].str.replace(',','').astype('float')
  df_bt['Volume'] = df_bt['Volume'].str.replace(',','').astype('float')

  class SmaCross1(bt.Strategy):
    params = dict(
      pfast=50, # period for the fast moving average
      pslow=200 # period for the slow moving average 
      ) 
    
    def __init__(self):
      sma1 = bt.ind.SMA(period = self.p.pfast) # fast moving average 
      sma2 = bt.ind.SMA(period = self.p.pslow) # slow moving average 
      self.crossover = bt.ind.CrossOver(sma1, sma2) # crossover signal 

    def next(self): 
      if not self.position: # not in the market 
        if self.crossover > 0: # if fast crosses slow to the upside 
          close = self.data.close[0] # 종가 값 
            
          size = int(self.broker.getcash() / close) # 최대 구매 가능 개수 
          self.buy(size=size) # 매수 size = 구매 개수 설정 
      elif self.crossover < 0: # in the market & cross to the downside 
          self.close() # 매도
          
  # 세레브로 (벡트레이더 엔진) 설정

  # 세레브로 가져오기
  cerebro = bt.Cerebro()

  # 데이터 가져오기
  data = bt.feeds.PandasData(dataname=df_bt,name="SAMSUNG")


  # 데이터 추가하기
  cerebro.adddata(data)

  # 전략 추가하기
  cerebro.addstrategy(SmaCross1)

  # 브로커 설정
  cerebro.broker.setcash (1000000)

  # 매매 단위 설정
  cerebro.addsizer(bt.sizers.SizerFix, stake = 30) #한번에 30주 설정.

  # 3. 세레브로 실행하기

  # 초기 투자금 가져오기
  init_cash = cerebro.broker.getvalue()

  # 세레브로 실행하기
  cerebro.run()

  # 최종 금액 가져오기
  final_cash = cerebro.broker.getvalue()

  print(i)
  print("최종금액 : ", final_cash, "원")
  print("수익률 : ", float(final_cash - init_cash)/float(init_cash) * 100., "%")
  print("\n")

KR7005930003
최종금액 :  1369350.0 원
수익률 :  36.935 %


KR7000660001
최종금액 :  1452130.0 원
수익률 :  45.213 %


KR7035420009
최종금액 :  15679500.0 원
수익률 :  1467.95 %


KR7207940008
최종금액 :  55000.0 원
수익률 :  -94.5 %


KR7005931001
최종금액 :  37760850.0 원
수익률 :  3676.0849999999996 %


KR7035720002
최종금액 :  1524800.0 원
수익률 :  52.480000000000004 %


KR7051910008
최종금액 :  2786300.0 원
수익률 :  178.63 %


KR7006400006
최종금액 :  2155400.0 원
수익률 :  115.53999999999999 %


KR7005380001
최종금액 :  1592450.0 원
수익률 :  59.245000000000005 %


KR7000270009
최종금액 :  95581050.0 원
수익률 :  9458.105 %




In [38]:
for i in mktcap_code:  

  print(i)
  
  # backtest 데이터 가공

  data = pd.read_json(json_data[i], orient ='index') 
  df = data.transpose()

  #시간순 재정렬
  df = df.sort_values(by=['TRD_DD'])
  df.reset_index(drop=True,inplace=True)
  df['TRD_DD'] = pd.to_datetime(df['TRD_DD']) #datetime변환

  #인풋 데이터 모양 맞춰주기(backtest에 들어갈 데이터 모양)
  df_bt = df[['TRD_DD','TDD_OPNPRC','TDD_HGPRC','TDD_LWPRC','TDD_CLSPRC', 'ACC_TRDVOL']]
  df_bt['TRD_DD'] = pd.to_datetime(df_bt['TRD_DD'])
  df_bt.rename(columns={'TRD_DD':'Date', 'TDD_OPNPRC':'Open', 'TDD_HGPRC':'High','TDD_LWPRC':'Low','TDD_CLSPRC':'Close', 'ACC_TRDVOL':'Volume'}, inplace=True)
  df_bt.set_index('Date',drop=True,inplace=True)

  #데이터프레임 콤마(,) 제거 그리고 타입 소수로 변환
  df_bt['Open'] = df_bt['Open'].str.replace(',','').astype('float')
  df_bt['High'] = df_bt['High'].str.replace(',','').astype('float')
  df_bt['Low'] = df_bt['Low'].str.replace(',','').astype('float')
  df_bt['Close'] = df_bt['Close'].str.replace(',','').astype('float')
  df_bt['Volume'] = df_bt['Volume'].str.replace(',','').astype('float')

  class SmaCross1(bt.Strategy):
    params = dict(
      pfast=50, # period for the fast moving average
      pslow=200 # period for the slow moving average 
      ) 
    
    def __init__(self):
      sma1 = bt.ind.SMA(period = self.p.pfast) # fast moving average 
      sma2 = bt.ind.SMA(period = self.p.pslow) # slow moving average 
      self.crossover = bt.ind.CrossOver(sma1, sma2) # crossover signal 

    def next(self): 
      if not self.position: # not in the market 
        if self.crossover > 0: # if fast crosses slow to the upside 
          close = self.data.close[0] # 종가 값 
            
          size = int(self.broker.getcash() / close) # 최대 구매 가능 개수 
          self.buy(size=size) # 매수 size = 구매 개수 설정 
      elif self.crossover < 0: # in the market & cross to the downside 
          self.close() # 매도

  random.seed(3)

  PARAM_NAMES = ["pfast", "pslow"]

  NGEN = 5  # 알고리즘 5번 반복.
  NPOP = 100 #인구 초기
  CXPB = 0.5  #교차 전략 
  MUTPB = 0.3  #돌연변이 전략.


  #최소fintness 설정 (fitness값이 작을수록 좋도록 설정)
  creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
  creator.create('Individual', list, fitness=creator.FitnessMin)

  # creator.create("FitnessMax", base.Fitness, weights=(1.0,))
  # creator.create("Individual", list, fitness=creator.FitnessMax)

  def evaluate(individual, plot=False, log=False):

    strategy_params = {k: v for k, v in zip(PARAM_NAMES, individual)}

    # 빠른 이동 평균은 느린 평균보다 느릴 수 없다
    # if strategy_params["pfast"] >= strategy_params["pslow"]:
    #     return [-np.inf]

    cerebro = bt.Cerebro(stdstats=False)

    data = bt.feeds.PandasData(dataname = df_bt, name = i)

    cerebro.adddata(data)

    initial_capital = 1000000
    cerebro.broker.setcash(initial_capital)

    cerebro.addstrategy(SmaCross1, **strategy_params)

    cerebro.addanalyzer(bt.analyzers.DrawDown)

    cerebro.broker.setcommission(commission=0.0025, margin=False)  #수수료 설정

    strats = cerebro.run()

    profit = cerebro.broker.getvalue() - initial_capital

    # max_dd = strats[0].analyzers.drawdown.get_analysis()["max"]["moneydown"] # max.moneydown - max drawdown value in monetary units
    # fitness = profit / (max_dd if max_dd > 0 else 1)
    fitness = round(1 / profit, 15)

    if log:
      print(f"Starting Portfolio Value: {initial_capital:,.2f}")
      print(f"Final Portfolio Value:  {cerebro.broker.getvalue():,.2f}")
      print(f"Total Profit:       {profit:,.2f}")
      print(f"Maximum Drawdown:     {max_dd:,.2f}")
      print(f"Profit / Max DD:     {fitness}")

    # if plot:
      # cerebro.plot()

    return [fitness]

  toolbox = base.Toolbox()
  toolbox.register("indices", random.sample, range(NPOP), NPOP)

  # crossover strategy
  toolbox.register("mate", tools.cxUniform, indpb=CXPB)
  # mutation strategy
  toolbox.register("mutate", tools.mutUniformInt, low=1, up=151, indpb=0.2)
  # selection strategy
  toolbox.register("select", tools.selTournament, tournsize=3)
  # fitness function
  toolbox.register("evaluate", evaluate)


  # definition of an individual & a population
  toolbox.register("attr_sma1", random.randint, 1, 100)
  toolbox.register("attr_sma2", random.randint, 151, 251) 
  toolbox.register(
    "individual",
    tools.initCycle,
    creator.Individual,
    (
      toolbox.attr_sma1,
      toolbox.attr_sma2,

    ),
  )

  toolbox.register("population", tools.initRepeat, list, toolbox.individual)

  mean = np.ndarray(NGEN)
  best = np.ndarray(NGEN)
  hall_of_fame = tools.HallOfFame(maxsize=3)

  t = time.perf_counter()
  pop = toolbox.population(n=NPOP)
  for g in trange(NGEN):
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
      if random.random() < CXPB:
        toolbox.mate(child1, child2)
        del child1.fitness.values
        del child2.fitness.values

    # Apply mutation on the offspring
    for mutant in offspring:
      if random.random() < MUTPB:
        toolbox.mutate(mutant)
        del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
      ind.fitness.values = fit

    # The population is entirely replaced by the offspring
    pop[:] = offspring
    hall_of_fame.update(pop)
    print(
      "HALL OF FAME:\n"
      + "\n".join(
        [
          f"  {_}: {ind}, Fitness: {ind.fitness.values[0]}"
          for _, ind in enumerate(hall_of_fame)
        ]
      )
    )

    fitnesses = [
      ind.fitness.values[0] for ind in pop if not np.isinf(ind.fitness.values[0])
    ]
    mean[g] = np.mean(fitnesses)
    best[g] = np.max(fitnesses)

  end_t = time.perf_counter()
  print(f"Time Elapsed: {end_t - t:,.2f}")

  # fig, ax = plt.subplots(sharex=True, figsize=(16, 9))

  # sns.lineplot(x=range(NGEN), y=mean, ax=ax, label="Average Fitness Score")
  # sns.lineplot(x=range(NGEN), y=best, ax=ax, label="Best Fitness Score")
  # ax.set_title("Fitness Score")
  # ax.set_xticks(range(NGEN))
  # ax.set_xlabel("Iteration")

  # plt.tight_layout()
  # plt.show()
  
  # 최적의 파라미터 값 출력
  OPTIMISED_STRATEGY_PARAMS = {
    k: v for k, v in zip(PARAM_NAMES, hall_of_fame[0])}
  op_params = list(OPTIMISED_STRATEGY_PARAMS.values())
  print('**파라미터 값: ', op_params)
  

  # 데이터 불러와서 가공
  df = pd.read_json(json_data[i], orient ='index') 
  read_df = df.transpose()
  read_df['TDD_CLSPRC'] = read_df['TDD_CLSPRC'].str.replace(',','').astype('float')
  read_df['pfast'] = talib.MA(read_df['TDD_CLSPRC'], timeperiod = op_params[0], matype=0)
  read_df['pslow'] = talib.MA(read_df['TDD_CLSPRC'], timeperiod = op_params[1], matype=0)
  read_df["GDC_sig"] = ""
  
  # 매도 매수 전략 설정 후 GDC_sig 열 추가
  first_cross = 0 
  for i in range(0, len(read_df)):
    if read_df['pfast'][i] < read_df['pslow'][i] and first_cross == 0:
      # print('Death cross on day', df['TRD_DD'][i], ':expect the price to continue to fall (매도)')
      read_df['GDC_sig'][i] = -1
      first_cross=1
    elif read_df['pfast'][i] > read_df['pslow'][i] and first_cross ==1:
      # print('Golden cross on day', df['TRD_DD'][i], ':expect the price to continue to rise (매수)')
      first_cross=0
      read_df['GDC_sig'][i] = 1
    else:
      read_df['GDC_sig'][i] = 0
  
  result = read_df.drop(['pfast', 'pslow'], axis='columns')
  print(result)


KR7005930003


/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
 20%|██        | 1/5 [03:57<15:49, 237.33s/it]

HALL OF FAME:
  0: [12, 195], Fitness: -0.00011447377835
  1: [95, 85], Fitness: -9.870292025e-06
  2: [71, 152], Fitness: -9.49954046e-06


 40%|████      | 2/5 [06:27<09:17, 185.79s/it]

HALL OF FAME:
  0: [12, 195], Fitness: -0.00011447377835
  1: [99, 224], Fitness: -1.26413061e-05
  2: [95, 85], Fitness: -9.870292025e-06


...

 80%|████████  | 4/5 [08:27<02:01, 121.58s/it]

HALL OF FAME:
  0: [95, 82], Fitness: -1.1890677115e-05
  1: [142, 129], Fitness: -5.021031847e-06
  2: [150, 125], Fitness: -1.737883692e-06


100%|██████████| 5/5 [10:12<00:00, 122.44s/it]

HALL OF FAME:
  0: [95, 82], Fitness: -1.1890677115e-05
  1: [150, 71], Fitness: -7.236845676e-06
  2: [150, 136], Fitness: -5.128001323e-06
Time Elapsed: 612.19
**파라미터 값:  [95, 82]


          TRD_DD  ISU_CD ISU_NM  TDD_CLSPRC  ...   PBR  DPS DVD_YLD GDC_sig
0     2021/12/09  035420  NAVER    399500.0  ...  8.91  402    0.10       0
1     2021/12/08  035420  NAVER    395000.0  ...  8.81  402    0.10       0
2     2021/12/07  035420  NAVER    391000.0  ...  8.72  402    0.10       0
3     2021/12/06  035420  NAVER    392000.0  ...  8.74  402    0.10       0
4     2021/12/03  035420  NAVER    402000.0  ...  8.96  402    0.10       0
...          ...     ...    ...         ...  ...   ...  ...     ...     ...
4724  2002/11/04  035420  엔에이치엔     41700.0  ...     -    -       -       0
4725  2002/11/01  035420  엔에이치엔     43900.0  ...     -    -       -       0
4726  2002/10/31  035420  엔에이치엔     46800.0  ...     -    -       -       0
4727  2002/10/30  035420  엔에이치엔     49250.0  ...     -    -       -       0
4728  2002/10/29  035420  엔에이치엔     44000.0  ...     -    -       -       0

[4729 rows x 16 columns]


In [ ]:
# json 파일 합치는 방법
# https://great-woman-hoseung.tistory.com/6